# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

I suggest you implement *constraint satisfaction* as it is described in the unit material. You can use the code you have previously been given as a guide. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%).

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [1]:
import numpy as np


def sudoku_solver(sudoku):
    solved_sudoku=[]
    allPossibleVals = []
    emptyPos=[]
    ### Checking if the array is valid
    for x in range (0,9):
        for y in range (0,9):
            if sudoku[x,y]!=0:
                if is_valid(sudoku,x,y) == False:
                    solved_sudoku = np.zeros((9,9))-1
                    return solved_sudoku
            else:
                emptyPos.append([x,y])
    invalid,allPossibleVals=update_all_possible(sudoku,emptyPos)
    
    changed=True
    rowdel=[[],[],[],[],[],[],[],[],[]]
    coldel=[[],[],[],[],[],[],[],[],[]]
    squaredel=[[],[],[],[],[],[],[],[],[]]

    
    while changed==True and invalid==False:
        invalid,allPossibleVals=update_all_possible(sudoku,emptyPos,rowdel,coldel,squaredel)
        changed,sudoku,emptyPos=set_all_singles(sudoku,emptyPos,allPossibleVals)
        if changed == True:
            continue

        sudoku,changed,rowdel,coldel,invalid=uniqueCandidate(sudoku,emptyPos,allPossibleVals,rowdel,coldel,invalid)
        emptyPos=UpdateEmpties(sudoku)
        if changed == True:
            continue

        rowdel,coldel,squaredel,changed,invalid=pairFinder(emptyPos,allPossibleVals,rowdel,coldel,squaredel)



    if invalid==True:
        solved_sudoku = np.zeros((9,9))-1
        return solved_sudoku

    
    solution=depthFirst(-1,emptyPos,allPossibleVals,sudoku,rowdel,coldel,squaredel)
    if solution == False:
        solved_sudoku = np.zeros((9,9))-1
        return solved_sudoku

    for x in range (0,9):
        for y in range (0,9):
            if sudoku[x,y]!=0:
                if is_valid(sudoku,x,y) == False:
                    solved_sudoku = np.zeros((9,9))-1
                    return solved_sudoku
    
    

    solved_sudoku=sudoku
    return solved_sudoku



def is_valid(aSudoku,x,y,valCheck=0,rowdel=[],coldel=[],squaredel=[]):


    bSudoku=aSudoku.copy()
    if valCheck == 0:
        sval=bSudoku[x,y]
    elif valCheck > 0:
        sval=valCheck
    row = bSudoku[x,:]
    row[y]=-10
    column = bSudoku[:,y]
    column[x]=-10

    sx=x - x%3
    sy=y - y%3
    square = bSudoku[ sx:sx+3 , sy:sy+3]
    square[x%3,y%3]=-10

    if sval in row or sval in column or sval in square:
        return False
    if len(rowdel)>0:
        for dels in rowdel[x]:
            if sval == dels[0] and [x,y] not in dels[1]:
                return False
    if len(coldel)>0:
        for dels in coldel[y]:
            if sval == dels[0] and [x,y] not in dels[1]:
                return False
    if len(squaredel)>0:
        for dels in squaredel[y-y%3+x//3]:
            if sval == dels[0] and [x,y] not in dels[1]:
                return False
    return True

def update_all_possible(sudoku,vacant,rowdel=[],coldel=[],squaredel=[]):

    mapPossible = []
    invalid=False
    for position in vacant:
        xPos=int(position[0])
        yPos=int(position[1])
        invalid,newlist=possible_vals(sudoku,xPos,yPos,rowdel,coldel,squaredel)
        if invalid:
            return invalid,mapPossible
        mapPossible.append(newlist)
    return invalid,mapPossible

def possible_vals(sudoku,x,y,rowdel=[],coldel=[],squaredel=[]):
    positionPossible =[]
    for checkval in range(1,10):
        if is_valid(sudoku,x,y,checkval,rowdel,coldel,squaredel):
            positionPossible.append(checkval)
    invalid=False
    if len(positionPossible)==0:
        invalid=True
            
    return invalid,positionPossible


def set_all_singles(sudoku,emptyPos,allPossibleVals):
    x=0
    y=0
    found=False
    emptyPoscopy=emptyPos.copy()
    for Vals in allPossibleVals:
        if len(Vals) == 1:
            sudoku[emptyPos[x][0],emptyPos[x][1]]=Vals[0]
            emptyPoscopy.pop(y)
            found=True
            #print("Single Set: ",emptyPos[x],Vals[0])
        else:
            y=y+1
        x=x+1
    return found,sudoku,emptyPoscopy

def uniqueCandidate(sudoku,emptyPos,allPossibleVals,rowdel,coldel,invalid):
    allSquares=[[],[],[],[],[],[],[],[],[]]
    sortedPos=[[],[],[],[],[],[],[],[],[]]

    b=False
    b=0
    sudokug=sudoku.copy()
    for x,y in emptyPos:
        try:
            allSquares[y-y%3+x//3].append(allPossibleVals[b].copy())
        except:
            return sudokug,True,rowdel,coldel,True
        sortedPos[y-y%3+x//3].append([x,y])
        b=b+1
    for k in range(0,9):
        if len(allSquares[k]) != 0:
            for j in range(1,10):
                eSet=[]
                for u in range(0,len(allSquares[k])):
                    if j in allSquares[k][u]:
                        eSet.append(sortedPos[k][u])
                if len(eSet)==1:##only 1 coordinate
                    sudokug[eSet[0][0],eSet[0][1]]=j
                    b=True
                    #print("Unique Candidate Found")
                elif len(eSet)==3:
                    if eSet[0][0] == eSet[1][0] and eSet[1][0] == eSet[2][0]:
                        if [j,eSet] not in rowdel[eSet[0][0]]:
                            rowdel[eSet[0][0]].append([j,eSet])
                            b=True
                            #print("tripple rowdel")
                    if eSet[0][1] == eSet[1][1] and eSet[1][1] == eSet[2][1]:
                        if [j,eSet] not in coldel[eSet[0][1]]:
                            coldel[eSet[0][1]].append([j,eSet])
                            b=True
                            #print("tripple coldel")
                elif len(eSet) == 2:
                    if eSet[0][0] == eSet[1][0]:
                        if [j,eSet] not in rowdel[eSet[0][0]]:
                            rowdel[eSet[0][0]].append([j,eSet])
                            b=True
                            #print("double rowdel")
                    if eSet[0][1] == eSet[1][1]:
                        if [j,eSet] not in coldel[eSet[0][1]]:
                            coldel[eSet[0][1]].append([j,eSet])
                            b=True
                            #print("double coldel")

                    

    return sudokug,b,rowdel,coldel,invalid




def UpdateEmpties(sudoku):
    emptyPos=[]
    for q in range(0,9):
        for w in range(0,9):
            if sudoku[q,w]==0:
                emptyPos.append([q,w])
    return emptyPos
    

def pairFinder(emptyPos,allPossibleVals,rowdel,coldel,squaredel):##double unique
    found=False
    invalid=False
    seen=[]
    seenindex=[]
    count=[]
    seen3=[]
    seen3index=[]
    seen3mode=[]

    
    for indexo in range(0,len(emptyPos)):
        if len(allPossibleVals[indexo])==3 and allPossibleVals[indexo] not in seen3:
            seen3.append(allPossibleVals[indexo])
            seen3index.append([emptyPos[indexo]])
            seen3mode.append([-1])
            count.append([0,0,0])
        elif len(allPossibleVals[indexo])==3:
            itsin=False
            g2=[i for i,val in enumerate(seen3) if val==allPossibleVals[indexo]]
            for g in g2:
                x1=seen3index[g][0][0]
                y1=seen3index[g][0][1]
                x2=emptyPos[indexo][0]
                y2=emptyPos[indexo][1]
                sq1=y1-y1%3+x1//3
                sq2=y2-y2%3+x2//3
                if x1==x2 or y1==y2 or sq1==sq2:
                    itsin=True
            if itsin==False:
                seen3.append(allPossibleVals[indexo])
                seen3index.append([emptyPos[indexo]])
                seen3mode.append([-1])
                count.append([0,0,0])
    

    for indexo in range(0,len(emptyPos)):


        if len(allPossibleVals[indexo]) == 2 and allPossibleVals[indexo] not in seen:
            seen.append(allPossibleVals[indexo])
            seenindex.append(emptyPos[indexo])
        elif allPossibleVals[indexo] in seen:
            uniqueVal=False
            g2=[i for i,val in enumerate(seen) if val==allPossibleVals[indexo]]
            for g in g2:
                x1=seenindex[g][0]
                y1=seenindex[g][1]
                x2=emptyPos[indexo][0]
                y2=emptyPos[indexo][1]
                
                if x1 == x2:
                    uniqueVal=True
                    if [seen[g][0],[seenindex[g],emptyPos[indexo]]] not in rowdel[x1]:
                        rowdel[x1].append([seen[g][0],[seenindex[g],emptyPos[indexo]]])
                        #print("pair rowdel 1")
                        found=True
                    if [seen[g][1],[seenindex[g],emptyPos[indexo]]] not in rowdel[x1]:
                        rowdel[x1].append([seen[g][1],[seenindex[g],emptyPos[indexo]]])
                        #print("pair rowdel 2")
                        found=True


                if y1 == y2:
                    uniqueVal=True
                    if [seen[g][0],[seenindex[g],emptyPos[indexo]]] not in coldel[y1]:
                        coldel[y1].append([seen[g][0],[seenindex[g],emptyPos[indexo]]])
                        #print("pair coldel 1")
                        found=True
                    if [seen[g][1],[seenindex[g],emptyPos[indexo]]] not in coldel[y1]:
                        coldel[y1].append([seen[g][1],[seenindex[g],emptyPos[indexo]]])
                        #print("pair coldel 2")
                        found=True
                
                sq1=y1-y1%3+x1//3
                sq2=y2-y2%3+x2//3
                
                if sq1==sq2:
                    uniqueVal=True
                    if [seen[g][0],[seenindex[g],emptyPos[indexo]]] not in squaredel[sq1]:
                        squaredel[sq1].append([seen[g][0],[seenindex[g],emptyPos[indexo]]])
                        #print("pair squaredel 1")
                        found=True
                    if [seen[g][1],[seenindex[g],emptyPos[indexo]]] not in squaredel[sq1]:
                        squaredel[sq1].append([seen[g][1],[seenindex[g],emptyPos[indexo]]])
                        #print("pair squaredel 2")
                        found=True

            if uniqueVal == False and len(allPossibleVals[indexo])==2:
                seen.append(allPossibleVals[indexo])
                seenindex.append(emptyPos[indexo])
        
        g2=[]
        inside=False
        if len(allPossibleVals[indexo]) == 2:
            for ui in range(0,len(seen3)):
                insidex=allPossibleVals[indexo][0] in seen3[ui]
                insidex=insidex and allPossibleVals[indexo][1] in seen3[ui]
                if insidex==True:
                    inside=True
                    g2.append(ui)
        else:
            inside=allPossibleVals[indexo] in seen3
            g2=[i for i,val in enumerate(seen3) if val==allPossibleVals[indexo]]


        if inside:
            for g in g2:
                x1=seen3index[g][0][0]
                y1=seen3index[g][0][1]
                x2=emptyPos[indexo][0]
                y2=emptyPos[indexo][1]
                if [x2,y2] not in seen3index[g]:
                    if x1 == x2:
                            count[g][0]=count[g][0]+1
                            seen3mode[g].append(0)
                            seen3index[g].append([x2,y2])

                    if y1 == y2:
                            count[g][1]=count[g][1]+1
                            seen3mode[g].append(1)
                            seen3index[g].append([x2,y2])
                    
                    sq1=y1-y1%3+x1//3
                    sq2=y2-y2%3+x2//3
                    
                    if sq1==sq2:
                            count[g][2]=count[g][2]+1
                            seen3mode[g].append(2)
                            seen3index[g].append([x2,y2])
        

        



    for indexo in range(0,len(count)):
        rowindexes=[seen3index[indexo][0]]
        colindexes=[seen3index[indexo][0]]
        squindexes=[seen3index[indexo][0]]

        for gen in range(0,len(seen3mode[indexo])):
            if seen3mode[indexo][gen]==0:
                rowindexes.append(seen3index[indexo][gen])
            if seen3mode[indexo][gen]==1:
                colindexes.append(seen3index[indexo][gen])
            if seen3mode[indexo][gen]==2:
                squindexes.append(seen3index[indexo][gen])


        x=seen3index[indexo][0][0]
        y=seen3index[indexo][0][1]
        sq=y-y%3+x//3

        if count[indexo][0] > 2 or count[indexo][1] > 2 or count[indexo][2] > 2:
            invalid = False
            #if any triple has more than 3 pairs then its invalid.
        if count[indexo][0]==2:
            if [seen3[indexo][0],rowindexes] not in rowdel[x]:
                rowdel[x].append([seen3[indexo][0],rowindexes])
                found=True
                #triple pair delete from row first number
            if [seen3[indexo][1],rowindexes] not in rowdel[x]:
                rowdel[x].append([seen3[indexo][1],rowindexes])
                found=True
                #triple pair delete from row second number
            if [seen3[indexo][2],rowindexes] not in rowdel[x]:
                rowdel[x].append([seen3[indexo][2],rowindexes])
                found=True
                #triple pair delete from row third number
            
        if count[indexo][1]==2:
            if [seen3[indexo][0],colindexes] not in coldel[y]:
                coldel[y].append([seen3[indexo][0],colindexes])
                found=True
                #triple pair delete from column first number
            if [seen3[indexo][1],colindexes] not in coldel[y]:
                coldel[y].append([seen3[indexo][1],colindexes])
                found=True
                #triple pair delete from column second number
            if [seen3[indexo][2],colindexes] not in coldel[y]:
                coldel[y].append([seen3[indexo][2],colindexes])
                found=True
                #triple pair delete from column third number
            
        if count[indexo][2]==2:
            if [seen3[indexo][0],squindexes] not in squaredel[sq]:
                squaredel[sq].append([seen3[indexo][0],squindexes])
                found=True
                #triple pair delete from square first number
            if [seen3[indexo][1],squindexes] not in squaredel[sq]:
                squaredel[sq].append([seen3[indexo][1],squindexes])
                found=True
                #triple pair delete from square second number
            if [seen3[indexo][2],squindexes] not in squaredel[sq]:
                squaredel[sq].append([seen3[indexo][2],squindexes])
                found=True
                #triple pair delete from square third number



    return rowdel,coldel,squaredel,found,invalid

def depthFirst(posIndex,emptyPos,allPossibleVals,sudoku,rowdel,coldel,squaredel):
    iteration=-1
    solution=False

    posIndex=posIndex+1
    if posIndex==len(allPossibleVals):
        return True
    for o in allPossibleVals[posIndex]:
        iteration=iteration+1
        if isinstance(o,int):
            q=o
        else:
            q=o[iteration]
        if is_valid(sudoku,emptyPos[posIndex][0],emptyPos[posIndex][1],q,rowdel,coldel,squaredel):
            sudoku[emptyPos[posIndex][0],emptyPos[posIndex][1]]=q
            solution=depthFirst(posIndex,emptyPos,allPossibleVals,sudoku,rowdel,coldel,squaredel)
            if solution==True:
                return True
            sudoku[emptyPos[posIndex][0],emptyPos[posIndex][1]]=0
    return False


All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [2]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.015625 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 

This is your solution for medium sudoku number 11
[[6 8 2 1 7 5 3 4 9]
 [9 5 3 4 8 6 7 2 1]
 [4 1 7 2 3 9 5 6 8]
 [3 9 8 6 1 7 4 5 2]
 [2 4 1 9 5 3 6 8 7]
 [5 7 6 8 2 4 1 9 3]
 [1 2 5 7 6 8 9 3 4]
 [7 3 4 5 9 2 8 1 6]
 [8 6 9 3 4 1 2 7 5]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.015625 seconds to solve.

This is medium sudoku number 12
[[0 4 6 0 0 9 3 0 2]
 [8 3 1 0 5 6 9 4 7]
 [0 9 7 3 4 1 8 0 5]
 [3 7 5 0 2 0 1 0 4]
 [0 6 2 4 0 3 5 0 8]
 [4 1 8 0 9 7 0 2 0]
 [1 2 9 8 3 4 7 0 6]
 [7 5 3 9 6 2 0 8 1]
 [6 8 4 1 7 5 2 0 0]]
This is your solution for medium sudoku number 12
[[5 4 6 7 8 9 3 1 2]
 [8 3 1 2 5 6 9 4 7]
 [2 9 7 3 4 1 8 6 5]
 [3 7 5 6 2 8 1 9 4]
 [9 6 2 4 1 3 5 7 8]
 [4 1 8 5 9 7 6 2 3]
 [1 2 9 8 3 4 7 5 6]
 [7 5 3 9 6 2 4 8 1]
 [6 8 4 1 7 5 2 3 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.0 seconds to solve.

This is medium sudoku number 13
[[0 7 8 0 5 2 4 0 3]
 [5 1 9 0 7 0 8 6 2]
 [0 4 3 8 0 6 5 7 1]
 [1 3 6 9 2 4 7 0 8]

This is your solution for hard sudoku number 11
[[9 1 7 5 2 6 3 8 4]
 [3 5 4 8 1 7 2 9 6]
 [6 2 8 4 3 9 5 7 1]
 [2 7 6 9 5 1 4 3 8]
 [1 3 5 2 4 8 9 6 7]
 [4 8 9 6 7 3 1 2 5]
 [5 9 2 7 6 4 8 1 3]
 [7 4 1 3 8 2 6 5 9]
 [8 6 3 1 9 5 7 4 2]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.203125 seconds to solve.

This is hard sudoku number 12
[[0 7 0 3 0 0 0 8 0]
 [5 8 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 9 4]
 [0 0 9 0 7 0 0 0 8]
 [0 0 5 0 8 0 0 0 2]
 [0 0 0 0 0 4 0 0 6]
 [0 0 0 0 5 0 0 0 0]
 [6 2 0 0 0 0 0 0 9]
 [0 0 0 0 6 7 0 1 0]]
This is your solution for hard sudoku number 12
[[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]]
Is your solution correct?
Yes! Correct solution.

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [6]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True;
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [ ]:
# This is a TEST CELL. Do not delete or change.